In [2]:
#imports
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as pl
import numpy as np
%matplotlib inline

Contaminated water can have many adverse health effects such as developmental effects to fetuses during pregnancy or to breastfed infants, kidney cancer, liver effects, immune effects, and thyroid effects. It is difficult to predict what may be causing the amount of days during which an area is under a contamination advisory. 

Data provided by the US Environmental Protection Agency includes data about each counties' water, land and population. There are many factors that may predict the amount of days per contamination advisory event, and first I will be determining if the number (in pounds) of herbicides, insecticides and fungicides applied has an affect on the amount of days per contamination advisory event. I will also go on to determine what other factors may predict contamination advisories, in order to find the most predictive feature. The objective here is to determine which of the features (either alone or combined) represented in this data set can most accurately predict the amount of days per contamination advisory. I will therefore be predicting a continuous number. 

While I do not have any direct experience working with water contamination issues, I am very interested in the ways we can improve the cleanliness of our water since it can affect huge populations. Research in this field often looks at storm water runoff which has been shown to increase water contamination since the land carries bacteria from sewage that can be carried into the water. The type of work done in a certain area can also have an impact on contamination, in that mining activities and burning fossil fuels can lead to acid rain. This data set includes features that show what percent of the population works in a particular industry, and it also includes the amount of chemicals present in a particular area's rainfall. 

The names of the features are convolted so I will need to rename the features that I want to work with. Also, some of the data was already log and or natural log transformed so I need to determine if this is an issue. I would also like to look at the relationships discussed in a statewide manner rather than just on an individual county level. 

One of the assumptions that I am making with this data is that the sampled population is representative of the true population. I have to assume this because I did not collect the data and I do not currently have the methods that were used to collect the data (but I can research this). Since this data was collected in 2005, I am assuming that 2005 was a year that is representative of all other years. However, something unknown may have happened this year that caused it to present abnormal data. 

I will consider this project a success if I am able to visualize a significant relationship (p-value < .05) between my features and the dependent variable (days per contamination event). I hope to find a combination of features that will predict the outcome more than any of the individual features. The features I will investigate besides pesticides will be the number of sewage permits per 1000km stream length, number of days per rain advisory event, mg/L of sulfates and nitrates present in the rainfall, acres irrigated per county acres (percentage), and proportion of all roads that are highways or secondary roads. 

Some of my first steps:

1. Visualize the relationship between the features and the response with scatterplots 
2. Find out how strong the relationships are 
3. Use cross validation to find a mean MSE and R2 for the model 
4. Use lasso and ridge regularization 
5. use grid search to find best alphas 
4. Use the model for prediction 
3. Find p-values for the relationships 

In [3]:
data = pd.read_csv("../dataset/EQIDATA_ALL_DOMAINS_2014MARCH11.csv")
print data.head()

   stfips     county_name state  fips  cat_rucc  a_teca_ln  a_112tca_ln  \
0    1001  Autauga County    AL  1001         1  -4.385941    -5.052391   
1    1003  Baldwin County    AL  1003         2  -4.385456   -14.249940   
2    1005  Barbour County    AL  1005         3  -4.943471   -10.852820   
3    1007     Bibb County    AL  1007         1  -5.952906    -9.329214   
4    1009   Blount County    AL  1009         1  -5.251164   -14.007340   

   a_dbcp_ln  a_tdi_ln  a_2clacephen_ln        ...         med_hh_value  \
0  -13.61030 -8.087206        -31.70556        ...                94800   
1  -13.61030 -8.208811        -16.06089        ...               122500   
2  -13.53285 -9.421164        -31.70556        ...                68600   
3  -13.61030 -9.428741        -16.09672        ...                74600   
4  -13.61030 -8.590843        -16.15294        ...                86800   

   med_hh_inc  pct_pers_lt_pov  pct_no_eng  pct_hs_more  pct_unemp  med_rooms  \
0       42013    

In [4]:
data.columns

Index([u'stfips', u'county_name', u'state', u'fips', u'cat_rucc', u'a_teca_ln',
       u'a_112tca_ln', u'a_dbcp_ln', u'a_tdi_ln', u'a_2clacephen_ln',
       ...
       u'med_hh_value', u'med_hh_inc', u'pct_pers_lt_pov', u'pct_no_eng',
       u'pct_hs_more', u'pct_unemp', u'med_rooms', u'work_out_co',
       u'pct_mt_10units_log', u'violent_rate_log'],
      dtype='object', length=227)

In [5]:
data.shape

(3141, 227)

In [ ]:

fig, axs = plt.subplots(1, 3, sharey=True)
data.plot(kind='scatter', x='', y='Sales', ax=axs[0], figsize=(16, 8))
data.plot(kind='scatter', x='Radio', y='Sales', ax=axs[1])
data.plot(kind='scatter', x='Newspaper', y='Sales', ax=axs[2])

In [7]:
print data['county_name'].isnull().sum()

0


In [15]:
print data['insecticides_ln'].isnull().sum()

0


In [16]:
print data['herbicides_ln'].isnull().sum()

0


In [17]:
print data['numDays_Cont_Activity_tot'].isnull().sum()

KeyError: 'numDays_Cont_Activity_tot'

In [9]:
data.columns

Index([u'stfips', u'county_name', u'state', u'fips', u'cat_rucc', u'a_teca_ln',
       u'a_112tca_ln', u'a_dbcp_ln', u'a_tdi_ln', u'a_2clacephen_ln',
       ...
       u'med_hh_value', u'med_hh_inc', u'pct_pers_lt_pov', u'pct_no_eng',
       u'pct_hs_more', u'pct_unemp', u'med_rooms', u'work_out_co',
       u'pct_mt_10units_log', u'violent_rate_log'],
      dtype='object', length=227)

In [10]:
data["pct_pers_lt_pov"]

0       10.9
1       10.1
2       26.8
3       20.6
4       11.7
5       33.5
6       24.6
7       16.1
8       17.0
9       15.6
10      15.7
11      24.5
12      22.6
13      17.1
14      13.9
15      14.7
16      14.0
17      26.6
18      14.9
19      18.4
20      22.1
21      13.0
22      15.1
23      31.1
24      15.4
25      10.2
26      20.9
27      15.7
28      17.3
29      18.9
        ... 
3111     9.9
3112     3.6
3113     2.7
3114     6.8
3115     9.1
3116     6.7
3117     6.5
3118    21.0
3119    14.1
3120     7.6
3121    12.9
3122    11.6
3123     9.1
3124    17.6
3125    13.9
3126    10.6
3127    10.1
3128     9.1
3129     9.0
3130    11.8
3131    13.4
3132    12.7
3133    11.7
3134    10.7
3135     9.7
3136     7.8
3137     6.0
3138     9.9
3139    14.1
3140     9.9
Name: pct_pers_lt_pov, dtype: float64